###### General Steps to Follow
1. Importing Packages
1. Criteria 1 - Angles
1. Criteria 2 - Angles
1. Criteria 3 - Angles
1. Criteria 4 - Angles
1. Full Test

# -------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

In [2]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# -------------------------------------------------------------------------------------------------------------

In [3]:
def evaluate(output, y_true):
    y_pred = np.zeros((output.shape[0],1), dtype = "int")
    y_pred[(output > 0.5)] = 1
    y_pred = y_pred.reshape(y_pred.shape[0])
    print("accuracy =", accuracy_score(y_true, y_pred))

## 2) Criteria 1 - Angles

#### Loading training data

In [4]:
criteria1_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_1.npy", allow_pickle = True)

In [5]:
len(criteria1_data_angles)

657

#### Getting X_train, X_test, y_train, y_test

In [7]:
X = np.vstack(criteria1_data_angles[:, 0])
y = np.vstack(criteria1_data_angles[:, 1])
X_train_criteria1_angles, X_test_criteria1_angles, y_train_criteria1_angles, y_test_criteria1_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [8]:
print("X_train_criteria1_angles shape:", X_train_criteria1_angles.shape)
print("y_train_criteria1_angles shape:", y_train_criteria1_angles.shape)
print("X_test_criteria1_angles shape:", X_test_criteria1_angles.shape)
print("y_test_criteria1_angles shape:", y_test_criteria1_angles.shape)

X_train_criteria1_angles shape: (525, 600, 8)
y_train_criteria1_angles shape: (525, 1)
X_test_criteria1_angles shape: (132, 600, 8)
y_test_criteria1_angles shape: (132, 1)


#### Building and training the model

In [9]:
criteria1_angles_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria1_angles.shape[1], X_train_criteria1_angles.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria1_angles_model"
)
criteria1_angles_model.summary()

Model: "criteria1_angles_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                11800     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-04-11 17:19:44.305179: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-04-11 17:19:44.305217: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: amroPC
2024-04-11 17:19:44.305230: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: amroPC
2024-04-11 17:19:44.305435: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.161.7
2024-04-11 17:19:44.305477: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.161.7
2024-04-11 17:19:44.305488: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.161.7


In [10]:
criteria1_angles_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [11]:
criteria1_angles_model.fit(X_train_criteria1_angles,y_train_criteria1_angles, epochs = 50)

Epoch 1/50
17/17 [==============================] - 5s 194ms/step - loss: 0.6454 - accuracy: 0.7733
Epoch 2/50
17/17 [==============================] - 3s 203ms/step - loss: 0.4876 - accuracy: 0.8190
Epoch 3/50
17/17 [==============================] - 3s 186ms/step - loss: 0.4803 - accuracy: 0.8190
Epoch 4/50
17/17 [==============================] - 3s 191ms/step - loss: 0.4741 - accuracy: 0.8190
Epoch 5/50
17/17 [==============================] - 3s 202ms/step - loss: 0.4772 - accuracy: 0.8190
Epoch 6/50
17/17 [==============================] - 4s 206ms/step - loss: 0.4757 - accuracy: 0.8190
Epoch 7/50
17/17 [==============================] - 4s 207ms/step - loss: 0.4748 - accuracy: 0.8190
Epoch 8/50
17/17 [==============================] - 4s 209ms/step - loss: 0.4757 - accuracy: 0.8190
Epoch 9/50
17/17 [==============================] - 4s 208ms/step - loss: 0.4776 - accuracy: 0.8190
Epoch 10/50
17/17 [==============================] - 4s 212ms/step - loss: 0.4746 - accuracy: 0.8190

KeyboardInterrupt: 

#### Saving the model

In [ ]:
criteria1_angles_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [12]:
y_pred = criteria1_angles_model.predict(X_train_criteria1_angles)
evaluate(y_pred, y_train_criteria1_angles)

17/17 [==============================] - 1s 64ms/step
accuracy = 0.819047619047619


###### Test Data

In [13]:
y_pred = criteria1_angles_model.predict(X_test_criteria1_angles)
evaluate(y_pred, y_test_criteria1_angles)

5/5 [==============================] - 0s 55ms/step
accuracy = 0.7575757575757576


# -------------------------------------------------------------------------------------------------------------

## 3) Criteria 2 - Poses

#### Loading training data

In [14]:
criteria2_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_2.npy", allow_pickle = True)

In [15]:
len(criteria2_data_angles)

764

#### Getting X_train, X_test, y_train, y_test

In [16]:
X = np.vstack(criteria2_data_angles[:, 0])
y = np.vstack(criteria2_data_angles[:, 1])
X_train_criteria2_angles, X_test_criteria2_angles, y_train_criteria2_angles, y_test_criteria2_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [17]:
print("X_train_criteria2_angles shape:", X_train_criteria2_angles.shape)
print("y_train_criteria2_angles shape:", y_train_criteria2_angles.shape)
print("X_test_criteria2_angles shape:", X_test_criteria2_angles.shape)
print("y_test_criteria2_angles shape:", y_test_criteria2_angles.shape)

X_train_criteria2_angles shape: (611, 600, 8)
y_train_criteria2_angles shape: (611, 1)
X_test_criteria2_angles shape: (153, 600, 8)
y_test_criteria2_angles shape: (153, 1)


#### Building and training the model

In [18]:
criteria2_angles_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria2_angles.shape[1], X_train_criteria2_angles.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria2_angles_model"
)
criteria2_angles_model.summary()

Model: "criteria2_angles_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                11800     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
criteria2_angles_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [20]:
criteria2_angles_model.fit(X_train_criteria2_angles,y_train_criteria2_angles, epochs = 50)

Epoch 1/50
20/20 [==============================] - 5s 178ms/step - loss: 0.6546 - accuracy: 0.7300
Epoch 2/50
20/20 [==============================] - 4s 205ms/step - loss: 0.5842 - accuracy: 0.7627
Epoch 3/50
20/20 [==============================] - 4s 182ms/step - loss: 0.5516 - accuracy: 0.7627
Epoch 4/50
20/20 [==============================] - 4s 202ms/step - loss: 0.5485 - accuracy: 0.7627
Epoch 5/50
20/20 [==============================] - 4s 180ms/step - loss: 0.5517 - accuracy: 0.7627
Epoch 6/50
20/20 [==============================] - 4s 175ms/step - loss: 0.5487 - accuracy: 0.7627
Epoch 7/50
20/20 [==============================] - 4s 199ms/step - loss: 0.5552 - accuracy: 0.7627
Epoch 8/50
20/20 [==============================] - 3s 166ms/step - loss: 0.5492 - accuracy: 0.7627
Epoch 9/50
20/20 [==============================] - 4s 182ms/step - loss: 0.5500 - accuracy: 0.7627
Epoch 10/50
20/20 [==============================] - 4s 184ms/step - loss: 0.5511 - accuracy: 0.7627

KeyboardInterrupt: 

#### Saving the model

In [ ]:
criteria2_angles_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [21]:
y_pred = criteria2_angles_model.predict(X_train_criteria2_angles)
evaluate(y_pred, y_train_criteria2_angles)

20/20 [==============================] - 2s 64ms/step
accuracy = 0.762684124386252


###### Test Data

In [23]:
y_pred = criteria2_angles_model.predict(X_test_criteria2_angles)
evaluate(y_pred, y_test_criteria2_angles)

5/5 [==============================] - 0s 65ms/step
accuracy = 0.7777777777777778


# -------------------------------------------------------------------------------------------------------------

## 4) Criteria 3 - Poses

#### Loading training data

In [24]:
criteria3_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_3.npy", allow_pickle = True)

In [25]:
len(criteria3_data_angles)

764

#### Getting X_train, X_test, y_train, y_test

In [26]:
X = np.vstack(criteria3_data_angles[:, 0])
y = np.vstack(criteria3_data_angles[:, 1])
X_train_criteria3_angles, X_test_criteria3_angles, y_train_criteria3_angles, y_test_criteria3_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [27]:
print("X_train_criteria3_angles shape:", X_train_criteria3_angles.shape)
print("y_train_criteria3_angles shape:", y_train_criteria3_angles.shape)
print("X_test_criteria3_angles shape:", X_test_criteria3_angles.shape)
print("y_test_criteria3_angles shape:", y_test_criteria3_angles.shape)

X_train_criteria3_angles shape: (611, 600, 8)
y_train_criteria3_angles shape: (611, 1)
X_test_criteria3_angles shape: (153, 600, 8)
y_test_criteria3_angles shape: (153, 1)


#### Building and training the model

In [29]:
criteria3_angles_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria3_angles.shape[1], X_train_criteria3_angles.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria3_angles_model"
)
criteria3_angles_model.summary()

Model: "criteria3_angles_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                11800     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
criteria3_angles_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [31]:
criteria3_angles_model.fit(X_train_criteria3_angles, y_train_criteria3_angles, epochs = 50)

Epoch 1/50
20/20 [==============================] - 5s 199ms/step - loss: 0.6934 - accuracy: 0.4534
Epoch 2/50
20/20 [==============================] - 4s 204ms/step - loss: 0.6933 - accuracy: 0.4812
Epoch 3/50
20/20 [==============================] - 4s 200ms/step - loss: 0.6932 - accuracy: 0.4926
Epoch 4/50
20/20 [==============================] - 4s 188ms/step - loss: 0.6933 - accuracy: 0.4959
Epoch 5/50
20/20 [==============================] - 4s 194ms/step - loss: 0.6932 - accuracy: 0.4975
Epoch 6/50
20/20 [==============================] - 4s 198ms/step - loss: 0.6932 - accuracy: 0.5041
Epoch 7/50
20/20 [==============================] - 4s 183ms/step - loss: 0.6934 - accuracy: 0.5041
Epoch 8/50
20/20 [==============================] - 4s 187ms/step - loss: 0.6932 - accuracy: 0.5041
Epoch 9/50
20/20 [==============================] - 4s 197ms/step - loss: 0.6932 - accuracy: 0.5041
Epoch 10/50
20/20 [==============================] - 4s 205ms/step - loss: 0.6932 - accuracy: 0.5041

KeyboardInterrupt: 

#### Saving the model

In [ ]:
criteria3_angles_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [32]:
y_pred = criteria3_angles_model.predict(X_train_criteria3_angles)
evaluate(y_pred, y_train_criteria3_angles)

20/20 [==============================] - 2s 63ms/step
accuracy = 0.5040916530278232


###### Test Data

In [33]:
y_pred = criteria3_angles_model.predict(X_test_criteria3_angles)
evaluate(y_pred, y_test_criteria3_angles)

5/5 [==============================] - 0s 46ms/step
accuracy = 0.5424836601307189


# -------------------------------------------------------------------------------------------------------------

## 5) Criteria 4 - Poses

#### Loading training data

In [34]:
criteria4_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_4.npy", allow_pickle = True)

In [35]:
len(criteria4_data_angles)

803

#### Getting X_train, X_test, y_train, y_test

In [36]:
X = np.vstack(criteria4_data_angles[:, 0])
y = np.vstack(criteria4_data_angles[:, 1])
X_train_criteria4_angles, X_test_criteria4_angles, y_train_criteria4_angles, y_test_criteria4_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [37]:
print("X_train_criteria4_angles shape:", X_train_criteria4_angles.shape)
print("y_train_criteria4_angles shape:", y_train_criteria4_angles.shape)
print("X_test_criteria4_angles shape:", X_test_criteria4_angles.shape)
print("y_test_criteria4_angles shape:", y_test_criteria4_angles.shape)

X_train_criteria4_angles shape: (642, 600, 8)
y_train_criteria4_angles shape: (642, 1)
X_test_criteria4_angles shape: (161, 600, 8)
y_test_criteria4_angles shape: (161, 1)


#### Building and training the model

In [38]:
criteria4_angles_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria4_angles.shape[1], X_train_criteria4_angles.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria4_angles_model"
)
criteria4_angles_model.summary()

Model: "criteria4_angles_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 50)                11800     
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
criteria4_angles_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [40]:
criteria4_angles_model.fit(X_train_criteria4_angles, y_train_criteria4_angles, epochs = 10)

Epoch 1/10
21/21 [==============================] - 5s 191ms/step - loss: 0.6852 - accuracy: 0.5857
Epoch 2/10
21/21 [==============================] - 3s 168ms/step - loss: 0.6678 - accuracy: 0.6199
Epoch 3/10
21/21 [==============================] - 4s 180ms/step - loss: 0.6645 - accuracy: 0.6199
Epoch 4/10
21/21 [==============================] - 4s 195ms/step - loss: 0.6654 - accuracy: 0.6199
Epoch 5/10
21/21 [==============================] - 4s 210ms/step - loss: 0.6643 - accuracy: 0.6199
Epoch 6/10
21/21 [==============================] - 4s 203ms/step - loss: 0.6659 - accuracy: 0.6199
Epoch 7/10
21/21 [==============================] - 4s 190ms/step - loss: 0.6653 - accuracy: 0.6199
Epoch 8/10
21/21 [==============================] - 4s 202ms/step - loss: 0.6660 - accuracy: 0.6199
Epoch 9/10
21/21 [==============================] - 4s 190ms/step - loss: 0.6651 - accuracy: 0.6199
Epoch 10/10
21/21 [==============================] - 4s 186ms/step - loss: 0.6658 - accuracy: 0.6199

#### Saving the model

In [ ]:
criteria4_angles_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [41]:
y_pred = criteria4_angles_model.predict(X_train_criteria4_angles)
evaluate(y_pred, y_train_criteria4_angles)

21/21 [==============================] - 2s 71ms/step
accuracy = 0.6199376947040498


###### Test Data

In [42]:
y_pred = criteria4_angles_model.predict(X_test_criteria4_angles)
evaluate(y_pred, y_test_criteria4_angles)

6/6 [==============================] - 0s 62ms/step
accuracy = 0.6024844720496895


# -------------------------------------------------------------------------------------------------------------

## 6) Full Test

In [43]:
def test_models(input_sample):
    input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
    y_pred1 = criteria1_angles_model.predict(input_sample)
    y_pred2 = criteria2_angles_model.predict(input_sample)
    y_pred3 = criteria3_angles_model.predict(input_sample)
    y_pred4 = criteria4_angles_model.predict(input_sample)
    print("Criteria1:", y_pred1[0][0])
    print("Criteria2:", y_pred2[0][0])
    print("Criteria3:", y_pred3[0][0])
    print("Criteria4:", y_pred4[0][0])

In [46]:
input_sample = X_train_criteria1_angles[3]
print(y_train_criteria1_angles[3])
test_models(input_sample)

[1]
1/1 [==============================] - 0s 26ms/step
Criteria1: 0.8185845
Criteria2: 0.7859407
Criteria3: 0.50251806
Criteria4: 0.37688214


In [47]:
input_sample = X_train_criteria2_angles[3]
print(y_train_criteria2_angles[3])
test_models(input_sample)

[0]
1/1 [==============================] - 0s 42ms/step
Criteria1: 0.8185845
Criteria2: 0.7859407
Criteria3: 0.50251806
Criteria4: 0.37688214


In [49]:
X_test_criteria4_angles[0][0:10]

array([[169.50684771, 162.84769609,   4.1882403 ,   2.727874  ,
          1.20038358,   5.19965741, 177.60078966, 176.75317561],
       [171.07256969, 164.39157358,   4.2995122 ,   2.58817305,
          0.29545303,   4.60142229, 177.60154118, 176.62097545],
       [171.35135647, 167.43512944,   4.10327848,   2.2068208 ,
          0.3588785 ,   3.53883359, 177.63417632, 176.84277062],
       [172.17654625, 169.05370578,   3.55601521,   2.73478632,
          0.46563795,   2.24025454, 177.53354121, 175.87431609],
       [173.16275894, 169.79950771,   2.9722592 ,   3.11384453,
          0.52651139,   1.49860221, 177.55019844, 175.28991665],
       [173.97695243, 169.68327281,   2.56359962,   2.86037265,
          0.49629203,   1.83846278, 177.37932429, 175.02798585],
       [171.49665878, 170.28643387,   2.43536232,   1.94571697,
          1.89200911,   2.47942379, 177.31506622, 175.18638695],
       [163.81347919, 169.48638103,   1.44002295,   1.32639016,
          6.82870787,   3.4960623

In [50]:
X_test_criteria4_angles[7][0:10]

array([[161.69119415, 159.49450875,  29.86415325,  33.51283843,
         38.86204192,  43.37978125, 161.91379261, 154.82482552],
       [161.01266507, 161.64893265,  31.03275206,  34.96794987,
         40.33972518,  43.70499578, 161.8373133 , 154.98182658],
       [161.97828832, 163.29445681,  31.81134444,  35.5104527 ,
         40.52502436,  43.34049907, 161.67674483, 155.06616292],
       [162.61365585, 164.37101358,  32.21248742,  35.77615241,
         40.56649217,  43.07691874, 161.62941808, 155.18650034],
       [162.63404781, 164.15335238,  32.43727959,  35.76221277,
         40.80454636,  43.17766152, 161.59573257, 155.25249239],
       [162.9243218 , 163.758957  ,  32.56075545,  35.7559155 ,
         40.78021475,  43.22850041, 161.55589275, 155.33308917],
       [161.98134785, 163.06187481,  32.84581272,  35.72758679,
         41.55943795,  43.45432341, 161.46287033, 155.41190651],
       [162.42946328, 162.79029342,  33.36970801,  35.9022371 ,
         41.8560012 ,  43.8502775